In [1]:
#importing pandas, numpy and sklearn for SVM analysis
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import StandardScaler

from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score

In [2]:
#Reading SalaryData_Train file, inspecting columns
salaryData=pd.read_csv('SalaryData_Train.csv')
salaryData=pd.DataFrame(salaryData)
salaryData.columns

Index(['age', 'workclass', 'education', 'educationno', 'maritalstatus',
       'occupation', 'relationship', 'race', 'sex', 'capitalgain',
       'capitalloss', 'hoursperweek', 'native', 'Salary'],
      dtype='object')

In [3]:
salaryData

,age,workclass,education,educationno,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native,Salary
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30156,27,Private,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
30157,40,Private,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
30158,58,Private,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
30159,22,Private,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [4]:
#For non numeric data, transforming data using dictionaries and map() function
salaryData["workclass"].drop_duplicates()
workClassDict={'State-gov':1,'Self-emp-not-inc':2,'Private':3,'Federal-gov':4,'Local-gov':5,'Self-emp-inc':6,'Without-pay':7}
salaryData['workclassInt'] = salaryData['workclass'].str.strip().map(workClassDict)

In [5]:
salaryData["education"].drop_duplicates()
educationDict={'Bachelors':1,'HS-grad':2,'11th':3,'Masters':4,'9th':5,'Some-college':6,'Assoc-acdm':7,'7th-8th':8,'Doctorate':9,'Assoc-voc':10,'Prof-school':11,'5th-6th':12,'10th':13,'Preschool':14,'12th':15,'1st-4th':16}
salaryData['educationInt'] = salaryData['education'].str.strip().map(educationDict)

In [6]:
salaryData["maritalstatus"].drop_duplicates()
maritalstatusDictv = {'Never-married':1,'Married-civ-spouse':2,'Divorced':3,'Married-spouse-absent':4,'Separated':5,'Married-AF-spouse':6,'Widowed':7}
salaryData['maritalstatusInt'] = salaryData['maritalstatus'].str.strip().map(maritalstatusDictv)

In [7]:
salaryData["occupation"].drop_duplicates()
occupationDict={'Adm-clerical':1,'Exec-managerial':2,'Handlers-cleaners':3,'Prof-specialty':4,'Other-service':5,'Sales':6,'Transport-moving':7,'Farming-fishing':8,'Machine-op-inspct':9,'Tech-support':10,'Craft-repair':11,'Protective-serv':12,'Armed-Forces':13,'Priv-house-serv':14}
salaryData['occupationInt'] = salaryData['occupation'].str.strip().map(occupationDict)

In [8]:
salaryData["relationship"].drop_duplicates()
relationshipDict={'Not-in-family':1,'Husband':2,'Wife':3,'Own-child':4,'Unmarried':5,'Other-relative':6}
salaryData['relationshipInt'] = salaryData['relationship'].str.strip().map(relationshipDict)

In [9]:
salaryData["race"].drop_duplicates()
raceDict = {'White':1,'Black':2,'Asian-Pac-Islander':3,'Amer-Indian-Eskimo':4,'Other':5}
salaryData['raceInt'] = salaryData['race'].str.strip().map(raceDict)

In [10]:
salaryData["sex"].drop_duplicates()
nativeDict = {'United-States':1,'Cuba':2,'Jamaica':3,'India':4,'Mexico':5,'Puerto-Rico':6,'Honduras':7,'England':8,'Canada':9,'Germany':10,'Iran':11,'Philippines':12,'Poland':13,'Columbia':14,'Cambodia':15,'Thailand':16,'Ecuador':17,'Laos':18,'Taiwan':19,'Haiti':20,'Portugal':21,'Dominican-Republic':22,'El-Salvador':23,'France':24,'Guatemala':25,'Italy':26,'China':27,'South':28,'Japan':29,'Yugoslavia':30,'Peru':31,'Outlying-US(Guam-USVI-etc)':32,'Scotland':33,'Trinadad&Tobago':34,'Greece':35,'Nicaragua':36,'Vietnam':37,'Hong':38,'Ireland':39,'Hungary':40}
salaryData['nativeInt'] = salaryData['native'].str.strip().map(nativeDict)

In [11]:
salaryData["Salary"].drop_duplicates()
SalaryDict  = {'<=50K':1,'>50K':2}
salaryData['SalaryInt'] = salaryData['Salary'].str.strip().map(SalaryDict)

In [12]:
salaryData["sex"].drop_duplicates()
SalaryDict  = {'Male':1,'Female':2}
salaryData['sexInt'] = salaryData['sex'].str.strip().map(SalaryDict)

In [13]:
X_train=salaryData[["age","workclassInt","educationInt","educationno","maritalstatusInt","occupationInt","relationshipInt","raceInt","sexInt","capitalgain","capitalloss","hoursperweek","nativeInt"]]
y_train=salaryData["SalaryInt"]

In [ ]:
clf = SVC()
#param_grid = [{'kernel':['linear'],'gamma':[50,5,0.5],'C':[15,13,10,0.1,0.001] }]
#gsv = GridSearchCV(clf,param_grid,cv=10)
clf = SVC(C= 15, gamma = 50, kernel='linear')
clf.fit(X_train , y_train)

In [ ]:
#gsv.fit(X_train,y_train)

In [ ]:
#gsv.best_params_ , gsv.best_score_ 

In [ ]:
#Reaading test file and transforming nin numeric data using map() function
import pandas as pd
salaryDataTest=pd.read_csv('SalaryData_Test.csv')
salaryDataTest=pd.DataFrame(salaryDataTest)

salaryDataTest['workclassInt'] = salaryDataTest['workclass'].str.strip().map(workClassDict)
salaryDataTest['educationInt'] = salaryDataTest['education'].str.strip().map(educationDict)
salaryDataTest['maritalstatusInt'] = salaryDataTest['maritalstatus'].str.strip().map(maritalstatusDictv)
salaryDataTest['occupationInt'] = salaryDataTest['occupation'].str.strip().map(occupationDict)
salaryDataTest['relationshipInt'] = salaryDataTest['relationship'].str.strip().map(relationshipDict)
salaryDataTest['raceInt'] = salaryDataTest['race'].str.strip().map(raceDict)
salaryDataTest['nativeInt'] = salaryDataTest['native'].str.strip().map(nativeDict)
salaryDataTest['SalaryInt'] = salaryDataTest['Salary'].str.strip().map(SalaryDict)
salaryDataTest['sexInt'] = salaryDataTest['sex'].str.strip().map(SalaryDict)



XTest=salaryDataTest[["age","workclassInt","educationInt","educationno","maritalstatusInt","occupationInt","relationshipInt","raceInt","sexInt","capitalgain","capitalloss","hoursperweek","nativeInt"]]
y_test = XTest.astype('float64', copy=False)

In [ ]:

y_pred = clf.predict(X_test)
acc = accuracy_score(y_test, y_pred) * 100
print("Accuracy =", acc)
confusion_matrix(y_test, y_pred)